# Voltage and Current Sources

This notebook covers all waveform types available in Pulsim for voltage and current sources.

## Contents
1. DC Sources
2. Pulse Waveform
3. Sine Waveform
4. PWL (Piecewise Linear) Waveform
5. PWM Waveform with Dead-Time
6. Current Sources

In [ ]:
import pulsim
import numpy as np
import matplotlib.pyplot as plt

# Helper function to run a quick simulation and plot
def simulate_and_plot(circuit, tstop, dt=1e-6, title=""):
    """Run simulation and plot all node voltages."""
    options = pulsim.SimulationOptions()
    options.tstop = tstop
    options.dt = dt
    
    result = pulsim.simulate(circuit, options)
    
    data = result.to_dict()
    time = np.array(data['time']) * 1e3  # Convert to ms
    
    plt.figure(figsize=(12, 4))
    for name, values in data['signals'].items():
        if name.startswith('V('):
            plt.plot(time, values, label=name, linewidth=1.5)
    
    plt.xlabel('Time (ms)')
    plt.ylabel('Voltage (V)')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    
    return result

## 1. DC Sources

The simplest source type - constant voltage or current.

```python
DCWaveform(value)  # value in Volts or Amps
```

In [ ]:
# DC Voltage Source Example
circuit = pulsim.Circuit()

# Add a 5V DC source
circuit.add_voltage_source("Vdc", "vout", "0", 5.0)

# Add a load resistor
circuit.add_resistor("R1", "vout", "0", 1000)  # 1kΩ

result = simulate_and_plot(circuit, tstop=1e-3, title="DC Voltage Source: 5V")

print(f"Output voltage: {np.mean(result.to_dict()['signals']['V(vout)']):.3f} V")

## 2. Pulse Waveform

Generates rectangular pulses with configurable parameters.

```python
PulseWaveform:
    v1      # Initial value (low level)
    v2      # Pulsed value (high level)
    td      # Delay time before first pulse
    tr      # Rise time
    tf      # Fall time
    pw      # Pulse width (time at v2)
    period  # Period of the waveform
```

```
        tr   pw    tf
       ┌──┐      ┌──┐
  v2 ──┤  │──────┤  │──
       │  │      │  │
  v1 ──┘  └──────┘  └──
       ├──────────────┤
           period
```

In [ ]:
# Create a pulse waveform
pulse = pulsim.PulseWaveform()
pulse.v1 = 0.0        # Low level: 0V
pulse.v2 = 5.0        # High level: 5V
pulse.td = 0.1e-3     # Delay: 0.1ms
pulse.tr = 10e-6      # Rise time: 10µs
pulse.tf = 10e-6      # Fall time: 10µs
pulse.pw = 0.4e-3     # Pulse width: 0.4ms
pulse.period = 1e-3   # Period: 1ms (1kHz)

print("Pulse Parameters:")
print(f"  Frequency: {1/pulse.period:.0f} Hz")
print(f"  Duty cycle: {pulse.pw/pulse.period*100:.0f}%")
print(f"  Rise/Fall time: {pulse.tr*1e6:.0f} µs")

In [ ]:
# Simulate pulse through RC filter
circuit = pulsim.Circuit()

# Pulse source - need to add via waveform variant
# For now, using JSON netlist approach
netlist = '''
{
  "name": "Pulse Source Demo",
  "components": [
    {"type": "V", "name": "Vpulse", "nodes": ["vin", "0"],
     "waveform": {"type": "pulse", "v1": 0, "v2": 5, "td": 0.1e-3,
                  "tr": 10e-6, "tf": 10e-6, "pw": 0.4e-3, "period": 1e-3}},
    {"type": "R", "name": "R1", "nodes": ["vin", "vout"], "value": 1000},
    {"type": "C", "name": "C1", "nodes": ["vout", "0"], "value": 100e-9}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 5e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

# Plot
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

time_ms = np.array(data['time']) * 1e3

axes[0].plot(time_ms, data['signals']['V(vin)'], 'b-', linewidth=1.5, label='V(vin)')
axes[0].set_ylabel('Input Voltage (V)')
axes[0].set_title('Pulse Waveform: 1kHz, 40% Duty Cycle')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(time_ms, data['signals']['V(vout)'], 'r-', linewidth=1.5, label='V(vout)')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Output Voltage (V)')
axes[1].set_title('RC Filtered Output (τ = 100µs)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## 3. Sine Waveform

Generates sinusoidal waveforms with optional damping.

```python
SineWaveform:
    offset     # DC offset
    amplitude  # Peak amplitude
    frequency  # Frequency in Hz
    delay      # Delay before start
    damping    # Exponential damping factor (theta)
```

$$v(t) = V_{offset} + V_{amplitude} \cdot \sin(2\pi f t) \cdot e^{-\theta t}$$

In [ ]:
# Create sine waveform parameters
sine = pulsim.SineWaveform()
sine.offset = 2.5       # DC offset: 2.5V
sine.amplitude = 2.0    # Peak amplitude: 2V
sine.frequency = 1000   # Frequency: 1kHz
sine.delay = 0          # No delay
sine.damping = 0        # No damping

print("Sine Parameters:")
print(f"  V(t) = {sine.offset} + {sine.amplitude}·sin(2π·{sine.frequency}·t)")
print(f"  Peak-to-peak: {2*sine.amplitude} V")
print(f"  Range: [{sine.offset - sine.amplitude}, {sine.offset + sine.amplitude}] V")

In [ ]:
# Simulate sine wave through RC low-pass filter
netlist = '''
{
  "name": "Sine Wave Demo",
  "components": [
    {"type": "V", "name": "Vsin", "nodes": ["vin", "0"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 5, 
                  "frequency": 1000, "delay": 0, "damping": 0}},
    {"type": "R", "name": "R1", "nodes": ["vin", "vout"], "value": 1000},
    {"type": "C", "name": "C1", "nodes": ["vout", "0"], "value": 1e-6}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 5e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

# Plot
time_ms = np.array(data['time']) * 1e3
v_in = np.array(data['signals']['V(vin)'])
v_out = np.array(data['signals']['V(vout)'])

plt.figure(figsize=(12, 5))
plt.plot(time_ms, v_in, 'b-', linewidth=1.5, label='V(in) - 1kHz Sine')
plt.plot(time_ms, v_out, 'r-', linewidth=1.5, label='V(out) - RC Filtered')

# Calculate attenuation
fc = 1 / (2 * np.pi * 1000 * 1e-6)  # Cutoff frequency
f = 1000  # Signal frequency
attenuation_theory = 1 / np.sqrt(1 + (f/fc)**2)

plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title(f'Sine Wave through RC Filter (fc = {fc:.0f} Hz, f = {f} Hz)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Measure attenuation
v_in_pp = np.max(v_in) - np.min(v_in)
v_out_pp = np.max(v_out[-1000:]) - np.min(v_out[-1000:])  # Steady-state
attenuation_sim = v_out_pp / v_in_pp

print(f"\nAttenuation at {f} Hz:")
print(f"  Theoretical: {attenuation_theory:.3f} ({20*np.log10(attenuation_theory):.1f} dB)")
print(f"  Simulated:   {attenuation_sim:.3f} ({20*np.log10(attenuation_sim):.1f} dB)")

In [ ]:
# Damped sine wave (for transient analysis)
netlist = '''
{
  "name": "Damped Sine Wave",
  "components": [
    {"type": "V", "name": "Vsin", "nodes": ["vout", "0"],
     "waveform": {"type": "sine", "offset": 0, "amplitude": 5, 
                  "frequency": 500, "delay": 0, "damping": 500}},
    {"type": "R", "name": "R1", "nodes": ["vout", "0"], "value": 1000}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 10e-3
options.dt = 10e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3
v_out = np.array(data['signals']['V(vout)'])

# Theoretical envelope
t = np.array(data['time'])
envelope = 5 * np.exp(-500 * t)

plt.figure(figsize=(12, 5))
plt.plot(time_ms, v_out, 'b-', linewidth=1.5, label='Damped Sine')
plt.plot(time_ms, envelope, 'r--', linewidth=1.5, label='Envelope')
plt.plot(time_ms, -envelope, 'r--', linewidth=1.5)

plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Damped Sine Wave: 5V·sin(2π·500·t)·e^(-500t)')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## 4. PWL (Piecewise Linear) Waveform

Defines arbitrary waveforms using time-value pairs.

```python
PWLWaveform:
    points  # List of (time, value) pairs
```

The output linearly interpolates between defined points.

In [ ]:
# Create a custom PWL waveform - triangle wave
pwl = pulsim.PWLWaveform()
pwl.points = [
    (0.0, 0.0),
    (0.5e-3, 5.0),   # Ramp up
    (1.0e-3, 0.0),   # Ramp down
    (1.5e-3, 5.0),
    (2.0e-3, 0.0),
    (2.5e-3, 5.0),
    (3.0e-3, 0.0),
]

print("PWL Points (Triangle Wave):")
for t, v in pwl.points:
    print(f"  t = {t*1e3:.1f} ms -> V = {v:.1f} V")

In [ ]:
# Simulate PWL waveform
netlist = '''
{
  "name": "PWL Waveform Demo",
  "components": [
    {"type": "V", "name": "Vpwl", "nodes": ["vin", "0"],
     "waveform": {"type": "pwl", "points": [
       [0.0, 0.0], [0.5e-3, 5.0], [1.0e-3, 0.0], 
       [1.5e-3, 5.0], [2.0e-3, 0.0], [2.5e-3, 5.0], [3.0e-3, 0.0]
     ]}},
    {"type": "R", "name": "R1", "nodes": ["vin", "vout"], "value": 1000},
    {"type": "C", "name": "C1", "nodes": ["vout", "0"], "value": 100e-9}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 3e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

axes[0].plot(time_ms, data['signals']['V(vin)'], 'b-', linewidth=1.5)
axes[0].set_ylabel('Input (V)')
axes[0].set_title('PWL Triangle Waveform')
axes[0].grid(True)

axes[1].plot(time_ms, data['signals']['V(vout)'], 'r-', linewidth=1.5)
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Output (V)')
axes[1].set_title('RC Filtered Output')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# PWL for custom arbitrary waveform - load profile
netlist = '''
{
  "name": "Load Profile",
  "components": [
    {"type": "V", "name": "Vload", "nodes": ["vout", "0"],
     "waveform": {"type": "pwl", "points": [
       [0.0, 0.0],
       [1e-3, 0.0],
       [1.1e-3, 3.3],
       [3e-3, 3.3],
       [3.1e-3, 5.0],
       [5e-3, 5.0],
       [5.1e-3, 1.0],
       [7e-3, 1.0],
       [7.1e-3, 0.0],
       [10e-3, 0.0]
     ]}},
    {"type": "R", "name": "R1", "nodes": ["vout", "0"], "value": 1000}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 10e-3
options.dt = 10e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

plt.figure(figsize=(12, 4))
plt.plot(time_ms, data['signals']['V(vout)'], 'g-', linewidth=2)
plt.fill_between(time_ms, data['signals']['V(vout)'], alpha=0.3)
plt.xlabel('Time (ms)')
plt.ylabel('Voltage (V)')
plt.title('Custom Load Profile using PWL')
plt.grid(True)
plt.tight_layout()
plt.show()

## 5. PWM Waveform with Dead-Time

Specialized waveform for power electronics applications.

```python
PWMWaveform:
    v_off         # Off-state voltage (typically 0V)
    v_on          # On-state voltage (gate drive, e.g., 12V)
    frequency     # Switching frequency (Hz)
    duty          # Duty cycle (0.0 to 1.0)
    dead_time     # Dead-time inserted at both edges (seconds)
    phase         # Phase offset (0.0 to 1.0, fraction of period)
    complementary # If true, output is inverted (for low-side)
```

### Dead-Time for Half-Bridge
```
High-side: ──┐     ┌────────────────┐     ┌──
             │     │                │     │
             └─────┘                └─────┘
                 ├─┤            ├─┤
              dead-time      dead-time

Low-side:  ──────┐     ┌────────────────┐     
                 │     │                │     
             ────┘     └────────────────┘─────
```

In [ ]:
# Create PWM waveform parameters
pwm = pulsim.PWMWaveform()
pwm.v_off = 0.0          # 0V when off
pwm.v_on = 12.0          # 12V gate drive
pwm.frequency = 100e3    # 100kHz switching
pwm.duty = 0.5           # 50% duty cycle
pwm.dead_time = 100e-9   # 100ns dead-time
pwm.phase = 0.0          # No phase offset
pwm.complementary = False

print("PWM Parameters:")
print(f"  Frequency: {pwm.frequency/1e3:.0f} kHz")
print(f"  Period: {pwm.period()*1e6:.1f} µs")
print(f"  Duty cycle: {pwm.duty*100:.0f}%")
print(f"  On-time: {pwm.t_on()*1e6:.1f} µs")
print(f"  Dead-time: {pwm.dead_time*1e9:.0f} ns")

In [ ]:
# Simulate PWM with complementary outputs
netlist = '''
{
  "name": "PWM Half-Bridge Demo",
  "components": [
    {"type": "V", "name": "Vhigh", "nodes": ["pwm_h", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 12, 
                  "frequency": 100e3, "duty": 0.5, "dead_time": 100e-9,
                  "phase": 0, "complementary": false}},
    {"type": "V", "name": "Vlow", "nodes": ["pwm_l", "0"],
     "waveform": {"type": "pwm", "v_off": 0, "v_on": 12, 
                  "frequency": 100e3, "duty": 0.5, "dead_time": 100e-9,
                  "phase": 0, "complementary": true}},
    {"type": "R", "name": "R1", "nodes": ["pwm_h", "0"], "value": 1000},
    {"type": "R", "name": "R2", "nodes": ["pwm_l", "0"], "value": 1000}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 30e-6  # 3 switching cycles
options.dt = 10e-9     # 10ns resolution for dead-time

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_us = np.array(data['time']) * 1e6

fig, axes = plt.subplots(3, 1, figsize=(12, 8), sharex=True)

# High-side PWM
axes[0].plot(time_us, data['signals']['V(pwm_h)'], 'b-', linewidth=1.5)
axes[0].set_ylabel('V_high (V)')
axes[0].set_title('High-Side Gate Drive')
axes[0].set_ylim([-1, 14])
axes[0].grid(True)

# Low-side PWM (complementary)
axes[1].plot(time_us, data['signals']['V(pwm_l)'], 'r-', linewidth=1.5)
axes[1].set_ylabel('V_low (V)')
axes[1].set_title('Low-Side Gate Drive (Complementary)')
axes[1].set_ylim([-1, 14])
axes[1].grid(True)

# Both overlaid to show dead-time
axes[2].plot(time_us, data['signals']['V(pwm_h)'], 'b-', linewidth=1.5, label='High-side')
axes[2].plot(time_us, data['signals']['V(pwm_l)'], 'r-', linewidth=1.5, label='Low-side')
axes[2].set_xlabel('Time (µs)')
axes[2].set_ylabel('Voltage (V)')
axes[2].set_title('Dead-Time Visualization (100ns gaps)')
axes[2].set_ylim([-1, 14])
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# PWM with different duty cycles
duty_cycles = [0.2, 0.5, 0.8]

fig, axes = plt.subplots(len(duty_cycles), 1, figsize=(12, 8), sharex=True)

for idx, duty in enumerate(duty_cycles):
    netlist = f'''
    {{
      "name": "PWM D={duty}",
      "components": [
        {{"type": "V", "name": "Vpwm", "nodes": ["pwm", "0"],
         "waveform": {{"type": "pwm", "v_off": 0, "v_on": 5, 
                      "frequency": 50e3, "duty": {duty}, "dead_time": 0}}}},
        {{"type": "R", "name": "R1", "nodes": ["pwm", "0"], "value": 1000}}
      ]
    }}
    '''
    
    circuit = pulsim.parse_netlist_string(netlist)
    options = pulsim.SimulationOptions()
    options.tstop = 60e-6
    options.dt = 100e-9
    
    result = pulsim.simulate(circuit, options)
    data = result.to_dict()
    
    time_us = np.array(data['time']) * 1e6
    
    axes[idx].plot(time_us, data['signals']['V(pwm)'], 'b-', linewidth=1.5)
    axes[idx].fill_between(time_us, data['signals']['V(pwm)'], alpha=0.3)
    axes[idx].set_ylabel('Voltage (V)')
    axes[idx].set_title(f'Duty Cycle = {duty*100:.0f}%')
    axes[idx].set_ylim([-0.5, 6])
    axes[idx].grid(True)

axes[-1].set_xlabel('Time (µs)')
plt.suptitle('PWM at 50kHz with Different Duty Cycles', y=1.02)
plt.tight_layout()
plt.show()

## 6. Current Sources

All waveform types also work with current sources.

```python
circuit.add_current_source(name, npos, nneg, value)
```

Current flows from `npos` to `nneg` (into `npos`, out of `nneg`).

In [ ]:
# DC Current Source
circuit = pulsim.Circuit()

# 1mA current source feeding a resistor
circuit.add_current_source("I1", "0", "vout", 1e-3)  # 1mA
circuit.add_resistor("R1", "vout", "0", 1000)  # 1kΩ

options = pulsim.SimulationOptions()
options.tstop = 1e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

v_out = np.mean(data['signals']['V(vout)'])
print(f"DC Current Source: 1mA into 1kΩ")
print(f"Expected voltage: V = I × R = 1mA × 1kΩ = 1V")
print(f"Simulated voltage: {v_out:.3f} V")

In [ ]:
# Pulsed current source - simulating load transient
netlist = '''
{
  "name": "Pulsed Current Load",
  "components": [
    {"type": "V", "name": "Vcc", "nodes": ["vcc", "0"], "value": 5.0},
    {"type": "R", "name": "Rsource", "nodes": ["vcc", "vout"], "value": 0.1},
    {"type": "C", "name": "Cbulk", "nodes": ["vout", "0"], "value": 100e-6},
    {"type": "I", "name": "Iload", "nodes": ["vout", "0"],
     "waveform": {"type": "pulse", "v1": 0.1, "v2": 2.0, "td": 1e-3,
                  "tr": 1e-6, "tf": 1e-6, "pw": 2e-3, "period": 5e-3}}
  ]
}
'''

circuit = pulsim.parse_netlist_string(netlist)

options = pulsim.SimulationOptions()
options.tstop = 10e-3
options.dt = 1e-6

result = pulsim.simulate(circuit, options)
data = result.to_dict()

time_ms = np.array(data['time']) * 1e3

fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# Calculate load current from capacitor equation
# For this example, we'll show voltage response
axes[0].plot(time_ms, data['signals']['V(vout)'], 'b-', linewidth=1.5)
axes[0].axhline(y=5.0, color='g', linestyle='--', label='Vcc = 5V')
axes[0].set_ylabel('Output Voltage (V)')
axes[0].set_title('Voltage Response to Load Transient')
axes[0].legend()
axes[0].grid(True)

# Show current waveform (reconstructed)
# Load current: 0.1A idle, 2A during pulse
t = np.array(data['time'])
i_load = np.where((t > 1e-3) & (t < 3e-3), 2.0, 
         np.where((t > 6e-3) & (t < 8e-3), 2.0, 0.1))

axes[1].plot(time_ms, i_load, 'r-', linewidth=1.5)
axes[1].fill_between(time_ms, i_load, alpha=0.3, color='red')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Load Current (A)')
axes[1].set_title('Pulsed Load Current (0.1A → 2A)')
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Calculate voltage droop
v_idle = data['signals']['V(vout)'][0]
v_min = np.min(data['signals']['V(vout)'])
droop = v_idle - v_min

print(f"\nLoad Transient Analysis:")
print(f"  Idle voltage: {v_idle:.3f} V")
print(f"  Minimum voltage: {v_min:.3f} V")
print(f"  Voltage droop: {droop*1e3:.1f} mV")

## Summary

Pulsim supports these waveform types for voltage and current sources:

| Waveform | Parameters | Use Case |
|----------|------------|----------|
| **DC** | value | Constant power supplies |
| **Pulse** | v1, v2, td, tr, tf, pw, period | Digital signals, square waves |
| **Sine** | offset, amplitude, frequency, delay, damping | AC analysis, oscillators |
| **PWL** | points (time, value pairs) | Arbitrary waveforms, load profiles |
| **PWM** | v_off, v_on, frequency, duty, dead_time, phase, complementary | Power electronics switching |

**Key Points:**
- All waveforms work with both voltage and current sources
- PWM has built-in dead-time and complementary output support
- PWL allows any arbitrary waveform shape
- Sine supports damping for transient analysis

**Next:** [Diodes and Semiconductor Models](06_diodes.ipynb)